In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd


/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
from backtesting.lib import SignalStrategy, TrailingStrategy


In [5]:
MSFT = pd.read_csv("/Users/fahad/project_repo/data/final/Moving_Average_Preparation/MSFT_MA_prep.csv", index_col=0)

In [7]:
MSFT.columns = ['Date', 'Open','High','Low','Close','Volume']

In [9]:
MSFT.Date = pd.to_datetime(MSFT.Date, format="%Y-%m-%d")

In [10]:
MSFT = MSFT.set_index(MSFT['Date'])

In [12]:
MSFT = MSFT.drop("Date", axis=1)


In [14]:
MSFT = MSFT[::-1]

In [32]:
MSFT.head()

,Open,High,Low,Close,Volume
2010-01-04,30.62,31.10,30.59,30.950,38409100
2010-01-05,30.85,31.10,30.64,30.960,49749600
2010-01-06,30.88,31.08,30.52,30.770,58182400
2010-01-07,30.63,30.70,30.19,30.452,50559700
2010-01-08,30.28,30.88,30.24,30.660,51197400


In [35]:
#dictionary to store equity
equity_dict = dict()

In [38]:
class SmaCrossTrailing(TrailingStrategy):
            
    n1 = 3
    n2 = 6
    
    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        self.buy()
        

            
#     def next(self):
#         #super().next()
#                 # If sma1 crosses above sma2, buy the asset
#         if crossover(self.sma1, self.sma2):
#             #print (Strategy.equity.__get__(self))
#             #print (self.data.index[-1])
#             equity_dict[self.data.index[-1]] = Strategy.equity.__get__(self)
#             self.buy()

#         # Else, if sma1 crosses below sma2, sell it
#         elif crossover(self.sma2, self.sma1):
#             #print (Strategy.equity.__get__(self))
#             #print (self.data.index[-1])
#             equity_dict[self.data.index[-1]] = Strategy.equity.__get__(self)
#             self.sell()
            

        
        

In [ ]:
bt = Backtest(MSFT, SmaCrossTrailing, cash=10000, commission=.002)

bt.run()

In [31]:
bt.plot(superimpose=False)

In [83]:
df = pd.DataFrame.from_dict([equity_dict])

In [84]:
df = df.T

In [85]:
df.columns = ['Equity']

In [86]:
df

,Equity
2010-10-29,10000.000000
2011-03-31,9426.830862
2011-07-29,8766.980344
2011-09-30,10000.000000
2011-10-31,8430.222915
2011-12-30,10645.079368
2012-01-31,7219.155141
2012-06-29,7314.045433
2013-03-28,7787.428500
2013-07-31,12456.077704


In [45]:
class SmaCross(Strategy):
    
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
#         print (self.sma1)
#         print (self.sma2)
    
    def next(self):
        # If sma1 crosses above sma2, buy the asset
        if crossover(self.sma1, self.sma2):
            #print (Strategy.equity.__get__(self))
            #print (self.data.index[-1])
            equity_dict[self.data.index[-1]] = Strategy.equity.__get__(self)
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif crossover(self.sma2, self.sma1):
            #print (Strategy.equity.__get__(self))
            #print (self.data.index[-1])
            equity_dict[self.data.index[-1]] = Strategy.equity.__get__(self)
            self.sell()